* Download libraries compatible to each others

In [ ]:
import subprocess
import sys
import os

def run_command(cmd):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"❌ Command failed: {cmd}")
            print(f"Stderr: {result.stderr}")
        else:
            print(f"✅ Command succeeded: {cmd}")
        return result.returncode == 0
    except Exception as e:
        print(f"Error running command: {e}")
        return False

print("🚀 Installing German Medical Chatbot Dependencies...")
print("="*60)


cleanup_commands = [
    "pip uninstall -y sentence-transformers huggingface_hub transformers tokenizers",
    "pip uninstall -y torch torchvision torchaudio"
]

for cmd in cleanup_commands:
    run_command(cmd)


install_commands = [
    "pip install --upgrade pip",
    "pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118",
    "pip install transformers>=4.35.0",
    "pip install faiss-cpu",
    "pip install PyPDF2",
    "pip install numpy",
    "pip install googletrans==4.0.0rc1",
    "pip install accelerate",
    "pip install scikit-learn",
    "pip install streamlit"
]


for cmd in install_commands:
    run_command(cmd)


print("✅ Installation complete!")
print("🔄 IMPORTANT: Please RESTART the runtime now!")
print("   Runtime > Restart runtime")
print("   Then run Cell 2")

🚀 Installing German Medical Chatbot Dependencies...
✅ Command succeeded: pip uninstall -y sentence-transformers huggingface_hub transformers tokenizers
✅ Command succeeded: pip uninstall -y torch torchvision torchaudio
✅ Command succeeded: pip install --upgrade pip
✅ Command succeeded: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
✅ Command succeeded: pip install transformers>=4.35.0
✅ Command succeeded: pip install faiss-cpu
✅ Command succeeded: pip install PyPDF2
✅ Command succeeded: pip install numpy
✅ Command succeeded: pip install googletrans==4.0.0rc1
✅ Command succeeded: pip install accelerate
✅ Command succeeded: pip install scikit-learn
✅ Command succeeded: pip install streamlit
✅ Installation complete!
🔄 IMPORTANT: Please RESTART the runtime now!
   Runtime > Restart runtime
   Then run Cell 2


* IMPORTS AND CAPABILITY DETECTION

In [ ]:
import torch
import os
import warnings
import numpy as np
import json
import pickle
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

print("🏥 German Medical Chatbot Starting...")
print("="*50)
print(f"✅ PyTorch {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

🏥 German Medical Chatbot Starting...
✅ PyTorch 2.7.1+cu118
✅ CUDA available: True


In [ ]:
TRANSFORMERS_AVAILABLE = False
ST_AVAILABLE = False
TRANSLATOR_AVAILABLE = False
FAISS_AVAILABLE = False

try:
    from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
    TRANSFORMERS_AVAILABLE = True
    print("✅ Transformers imported successfully!")
except Exception as e:
    print(f"⚠️ Transformers failed: {e}")

try:
    result = subprocess.run("pip install sentence-transformers==2.1.0", shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        from sentence_transformers import SentenceTransformer
        ST_AVAILABLE = True
        print("✅ SentenceTransformers installed and imported!")
    else:
        raise ImportError("Installation failed")
except Exception as e:
    print(f"⚠️ SentenceTransformers not available: {e}")

try:
    import faiss
    FAISS_AVAILABLE = True
    print("✅ FAISS imported successfully!")
except ImportError as e:
    print(f"⚠️ FAISS failed: {e}")

try:
    import PyPDF2
    print("✅ PyPDF2 imported successfully!")
except ImportError as e:
    print(f"❌ PyPDF2 failed: {e}")

try:
    from googletrans import Translator
    TRANSLATOR_AVAILABLE = True
    print("✅ GoogleTrans imported successfully!")
except ImportError as e:
    print(f"⚠️ GoogleTrans failed: {e}")

try:
    from google.colab import drive
    print("✅ Google Colab drive imported successfully!")
except ImportError:
    print("⚠️ Not running in Google Colab")

print(f"\n📊 System Capabilities:")
print(f"   🤖 Transformers: {'✅' if TRANSFORMERS_AVAILABLE else '❌'}")
print(f"   🔢 SentenceTransformers: {'✅' if ST_AVAILABLE else '❌'}")
print(f"   🔍 FAISS: {'✅' if FAISS_AVAILABLE else '❌'}")
print(f"   🌐 German Translation: {'✅' if TRANSLATOR_AVAILABLE else '❌'}")

✅ Transformers imported successfully!
⚠️ SentenceTransformers not available: cannot import name 'cached_download' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py)
✅ FAISS imported successfully!
✅ PyPDF2 imported successfully!
✅ GoogleTrans imported successfully!
✅ Google Colab drive imported successfully!

📊 System Capabilities:
   🤖 Transformers: ✅
   🔢 SentenceTransformers: ❌
   🔍 FAISS: ✅
   🌐 German Translation: ✅


* GOOGLE DRIVE SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/GermanMedicalChatbot"
os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/models", exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/documents", exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/vectordb", exist_ok=True)

print(f"✅ Project directory created: {PROJECT_DIR}")

Mounted at /content/drive
✅ Project directory created: /content/drive/MyDrive/GermanMedicalChatbot


* EMBEDDING CLASSES

In [ ]:
class TFIDFEmbedder:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            stop_words='english',
            ngram_range=(1, 2),
            min_df=1,
            max_df=0.95
        )
        self.is_fitted = False

    def fit_transform(self, texts):
        """ Fit and transform texts to embeddings"""
        embeddings = self.vectorizer.fit_transform(texts)
        self.is_fitted = True
        return embeddings.toarray().astype('float32')

    def transform(self, texts):
        """ Transform new texts to embeddings"""
        if not self.is_fitted:
            raise ValueError("Must fit vectorizer first")
        embeddings = self.vectorizer.transform(texts)
        return embeddings.toarray().astype('float32')

class SimpleEmbedder:
    def __init__(self, embedding_dim=384):
        self.embedding_dim = embedding_dim

    def encode(self, texts):
        embeddings = []
        for text in texts:
            words = text.lower().split()
            embedding = np.zeros(self.embedding_dim)
            for word in words[:self.embedding_dim]:
                embedding[hash(word) % self.embedding_dim] += 1.0
            if np.linalg.norm(embedding) > 0:
                embedding = embedding / np.linalg.norm(embedding)
            embeddings.append(embedding)
        return np.array(embeddings, dtype='float32')

* RAG SYSTEM CLASS

In [ ]:
class GermanMedicalRAG:


    def __init__(self, project_dir):
        self.project_dir = project_dir
        self.documents = []
        self.embeddings = []
        self.index = None
        self.embedding_model = None
        self.tfidf_embedder = TFIDFEmbedder()
        self.simple_embedder = SimpleEmbedder()

        if TRANSLATOR_AVAILABLE:
            try:
                self.translator = Translator()
                print("✅ German translator initialized")
            except:
                self.translator = None
                print("⚠️ Translator initialization failed")
        else:
            self.translator = None

    def load_embedding_model(self):
        if ST_AVAILABLE:
            try:
                model_path = f"{self.project_dir}/models/embeddings"
                if os.path.exists(model_path) and os.listdir(model_path):
                    print("📁 Loading embeddings from Google Drive...")
                    self.embedding_model = SentenceTransformer(model_path)
                else:
                    print("⬇️ Downloading embedding model...")
                    self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
                    os.makedirs(model_path, exist_ok=True)
                    self.embedding_model.save(model_path)
                    print("💾 Embedding model saved!")
                return True
            except Exception as e:
                print(f"❌ SentenceTransformer error: {e}")

        print("📝 Using TF-IDF embeddings")
        return False

    def process_pdf(self, pdf_path):
        text = ""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
        except Exception as e:
            print(f"Error processing PDF {pdf_path}: {e}")
        return text

    def chunk_text(self, text, chunk_size=1000, overlap=200):
        chunks = []
        words = text.split()
        for i in range(0, len(words), chunk_size - overlap):
            chunk = " ".join(words[i:i + chunk_size])
            if chunk.strip():
                chunks.append(chunk.strip())
        return chunks

    def add_documents(self, pdf_files):
        all_texts = []

        for pdf_file in pdf_files:
            pdf_path = f"{self.project_dir}/documents/{pdf_file.name}"
            with open(pdf_path, "wb") as f:
                f.write(pdf_file.getvalue())

            text = self.process_pdf(pdf_path)
            if not text.strip():
                print(f"⚠️ No text extracted from {pdf_file.name}")
                continue

            chunks = self.chunk_text(text)
            for chunk in chunks:
                all_texts.append({
                    'text': chunk,
                    'source': pdf_file.name
                })

        if not all_texts:
            raise Exception("No text extracted from any PDF files")

        texts_only = [doc['text'] for doc in all_texts]

        if self.embedding_model is not None:
            print("🔢 Creating embeddings with SentenceTransformers...")
            embeddings = self.embedding_model.encode(texts_only)
            embeddings = np.array(embeddings, dtype='float32')
        else:
            print("🔢 Creating embeddings with TF-IDF...")
            embeddings = self.tfidf_embedder.fit_transform(texts_only)

        self.documents = all_texts
        self.embeddings = embeddings

        if FAISS_AVAILABLE:
            normalized_embeddings = embeddings.copy()
            norms = np.linalg.norm(normalized_embeddings, axis=1, keepdims=True)
            norms[norms == 0] = 1
            normalized_embeddings = normalized_embeddings / norms

            dimension = embeddings.shape[1]
            self.index = faiss.IndexFlatIP(dimension)
            self.index.add(normalized_embeddings)
            faiss.write_index(self.index, f"{self.project_dir}/vectordb/faiss.index")

        with open(f"{self.project_dir}/vectordb/documents.pkl", "wb") as f:
            pickle.dump(self.documents, f)
        np.save(f"{self.project_dir}/vectordb/embeddings.npy", embeddings)

        print(f"✅ Processed {len(all_texts)} document chunks from {len(pdf_files)} PDFs")
        return len(all_texts)

    def load_existing_data(self):
        try:
            docs_path = f"{self.project_dir}/vectordb/documents.pkl"
            embeddings_path = f"{self.project_dir}/vectordb/embeddings.npy"

            if os.path.exists(docs_path) and os.path.exists(embeddings_path):
                with open(docs_path, "rb") as f:
                    self.documents = pickle.load(f)
                self.embeddings = np.load(embeddings_path)

                if FAISS_AVAILABLE:
                    index_path = f"{self.project_dir}/vectordb/faiss.index"
                    if os.path.exists(index_path):
                        self.index = faiss.read_index(index_path)

                self.load_embedding_model()
                print(f"✅ Loaded {len(self.documents)} existing document chunks")
                return True
        except Exception as e:
            print(f"Could not load existing data: {e}")
        return False

    def search_documents(self, query, k=3):
        if not self.documents:
            return []

        try:
            if self.embedding_model is not None:
                query_embedding = self.embedding_model.encode([query])
                query_embedding = np.array(query_embedding, dtype='float32')
            elif self.tfidf_embedder.is_fitted:
                query_embedding = self.tfidf_embedder.transform([query])
            else:
                query_embedding = self.simple_embedder.encode([query])

            if FAISS_AVAILABLE and self.index is not None:
                norm = np.linalg.norm(query_embedding)
                if norm > 0:
                    query_embedding = query_embedding / norm
                scores, indices = self.index.search(query_embedding, k)

                results = []
                for score, idx in zip(scores[0], indices[0]):
                    if idx != -1 and idx < len(self.documents):
                        results.append({
                            'text': self.documents[idx]['text'],
                            'source': self.documents[idx]['source'],
                            'score': float(score)
                        })
                return results
            else:
                similarities = cosine_similarity(query_embedding, self.embeddings)[0]
                top_indices = np.argsort(similarities)[::-1][:k]

                results = []
                for idx in top_indices:
                    results.append({
                        'text': self.documents[idx]['text'],
                        'source': self.documents[idx]['source'],
                        'score': float(similarities[idx])
                    })
                return results

        except Exception as e:
            print(f"Search error: {e}")
            return []

    def translate_text(self, text, target_lang='en'):
        if not self.translator:
            return text, 'en'

        try:
            if target_lang == 'auto':
                detected = self.translator.detect(text)
                return text, detected.lang
            else:
                result = self.translator.translate(text, dest=target_lang)
                return result.text, result.src
        except Exception as e:
            print(f"Translation error: {e}")
            return text, 'en'

* MODEL LOADING

In [ ]:
def load_model_safely():
    if not TRANSFORMERS_AVAILABLE:
        print("❌ Transformers not available")
        return None, None

    try:
        print("⬇️ Loading GPT-2 model...")
        tokenizer = AutoTokenizer.from_pretrained("gpt2")

        try:
            model = AutoModelForCausalLM.from_pretrained(
                "gpt2",
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None
            )
        except Exception as e:
            print(f"Device mapping failed, using basic loading: {e}")
            model = AutoModelForCausalLM.from_pretrained("gpt2")

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        print("✅ Language model loaded successfully!")
        return model, tokenizer

    except Exception as e:
        print(f"❌ Model loading failed: {e}")
        return None, None

model, tokenizer = load_model_safely()

⬇️ Loading GPT-2 model...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Language model loaded successfully!


* GERMAN MEDICAL CHATBOT CLASS

In [ ]:
class GermanMedicalChatbot:
    def __init__(self, model, tokenizer, rag_system):
        self.model = model
        self.tokenizer = tokenizer
        self.rag = rag_system
        self.generator = None

        if model and tokenizer:
            try:
                self.generator = pipeline(
                    "text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens=150,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )
                print("✅ Text generation pipeline created")
            except Exception as e:
                print(f"⚠️ Pipeline creation failed: {e}")
                print("📝 Will use template responses")

    def generate_response(self, user_input, language='en'):
        try:
            if language == 'de' and self.rag.translator:
                english_input, _ = self.rag.translate_text(user_input, 'en')
            else:
                english_input = user_input

            relevant_docs = self.rag.search_documents(english_input, k=3)
            context = ""
            sources = []
            if relevant_docs:
                for doc in relevant_docs:
                    context += f"{doc['text'][:400]}...\n\n"
                    sources.append(doc['source'])

            if context.strip():
                response = f"""Based on medical information about '{user_input}':

{context[:600]}

This information is for educational purposes only. Always consult healthcare professionals."""
            else:
                response = f"""I don't have specific information about '{user_input}' in my knowledge base.

Recommendations:
• Consult your healthcare provider
• Visit reputable medical websites
• Contact medical helplines

This chatbot provides information only, not medical advice."""

            if self.generator:
                try:
                    prompt = f"Medical question: {english_input}\nHelpful response:"
                    generated = self.generator(prompt, max_new_tokens=100)
                    llm_text = generated[0]['generated_text']
                    if "Helpful response:" in llm_text:
                        enhancement = llm_text.split("Helpful response:")[-1].strip()
                        if len(enhancement) > 30:
                            response = enhancement + "\n\n⚠️ For educational purposes only."
                except Exception as e:
                    pass

            if sources:
                response += f"\n\n📚 Sources: {', '.join(set(sources))}"
            if language == 'de' and self.rag.translator:
                response, _ = self.rag.translate_text(response, 'de')

            return response

        except Exception as e:
            error_msg = "I apologize for the error. Please consult a healthcare professional directly."
            if language == 'de' and self.rag.translator:
                error_msg, _ = self.rag.translate_text(error_msg, 'de')
            return error_msg

In [ ]:
print("🔧 Initializing German Medical RAG System...")
rag_system = GermanMedicalRAG(PROJECT_DIR)
rag_system.load_embedding_model()
if not rag_system.load_existing_data():
    print("📄 No existing documents found. Ready for PDF upload.")

print("🤖 Initializing German Medical Chatbot...")
chatbot = GermanMedicalChatbot(model, tokenizer, rag_system)

print("✅ German Medical Assistant ready!")

🔧 Initializing German Medical RAG System...
✅ German translator initialized
📝 Using TF-IDF embeddings


Device set to use cuda:0


📝 Using TF-IDF embeddings
✅ Loaded 2 existing document chunks
🤖 Initializing German Medical Chatbot...
✅ Text generation pipeline created
✅ German Medical Assistant ready!


* HELPER FUNCTIONS

In [ ]:
def upload_medical_pdfs():
    from google.colab import files

    print("📁 Please select your medical PDF files:")
    uploaded = files.upload()

    if uploaded:
        pdf_files = []
        for filename, content in uploaded.items():
            if filename.endswith('.pdf'):
                class SimpleFile:
                    def __init__(self, name, content):
                        self.name = name
                        self.content = content
                    def getvalue(self):
                        return self.content
                pdf_files.append(SimpleFile(filename, content))

        if pdf_files:
            try:
                result = rag_system.add_documents(pdf_files)
                print(f"✅ Successfully processed {len(pdf_files)} PDFs into {result} chunks!")
                return True
            except Exception as e:
                print(f"❌ Error processing PDFs: {e}")
                return False
        else:
            print("❌ No PDF files found in upload.")
            return False
    else:
        print("❌ No files uploaded.")
        return False

def chat_german(question):
    print(f"🇩🇪 Frage: {question}")

    try:
        english_question, detected_lang = rag_system.translate_text(question, 'en')
        print(f"📝 Übersetzt zu Englisch: {english_question}")

        relevant_docs = rag_system.search_documents(english_question, k=3)

        context = ""
        sources = []
        if relevant_docs:
            for doc in relevant_docs:
                context += f"{doc['text'][:400]}...\n\n"
                sources.append(doc['source'])

        if context.strip():
            english_response = f"""Based on medical information about '{question}':

{context[:600]}

This information is for educational purposes only. Always consult healthcare professionals."""
        else:
            english_response = f"""I don't have specific information about '{question}' in my knowledge base.

Recommendations:
• Consult your healthcare provider
• Visit reputable medical websites
• Contact medical helplines

This chatbot provides information only, not medical advice."""

        print(f"📝 Englische Antwort erstellt...")
        german_response, _ = rag_system.translate_text(english_response, 'de')

        if sources:
            source_text = f"\n\n📚 Quellen: {', '.join(set(sources))}"
            german_response += source_text

        print(f"🤖 Deutsche Antwort: {german_response}")
        return german_response

    except Exception as e:
        error_msg = "Entschuldigung, es gab einen Fehler. Bitte konsultieren Sie einen Arzt direkt."
        print(f"🤖 Fehler-Antwort: {error_msg}")
        return error_msg

def chat_english(question):
    return chatbot.generate_response(question, language='en')

def system_status():
    print("🔍 System Status:")
    print(f"   🤖 Transformers: {'✅' if TRANSFORMERS_AVAILABLE else '❌'}")
    print(f"   🔢 SentenceTransformers: {'✅' if ST_AVAILABLE else '❌'}")
    print(f"   🔍 FAISS: {'✅' if FAISS_AVAILABLE else '❌'}")
    print(f"   🌐 German Translation: {'✅' if TRANSLATOR_AVAILABLE else '❌'}")
    print(f"   📚 Documents: {len(rag_system.documents)} chunks loaded")
    print(f"   🎯 LLM Pipeline: {'✅' if chatbot.generator else '❌'}")
    print(f"   💾 Embedding Method: {'SentenceTransformers' if ST_AVAILABLE else 'TF-IDF'}")

def test_german_system():
    print("🧪 Testing German Medical System...")
    print("="*50)

    if len(rag_system.documents) == 0:
        print("⚠️ No documents loaded. Please upload medical PDFs first!")
        print("Run: upload_medical_pdfs()")
        return

    test_questions = [
        "Was ist Diabetes?",
        "Wie viele Menschen haben Diabetes?",
        "Was ist Hyperglykämie?",
        "Wie hoch ist die Diabetes-Rate bei Afroamerikanern?"
    ]

    for i, question in enumerate(test_questions, 1):
        print(f"\n{'='*30} Test {i} {'='*30}")
        chat_german(question)


In [ ]:
system_status()

🔍 System Status:
   🤖 Transformers: ✅
   🔢 SentenceTransformers: ❌
   🔍 FAISS: ✅
   🌐 German Translation: ✅
   📚 Documents: 2 chunks loaded
   🎯 LLM Pipeline: ✅
   💾 Embedding Method: TF-IDF


In [ ]:
upload_medical_pdfs()

📁 Please select your medical PDF files:


TypeError: 'NoneType' object is not subscriptable

In [ ]:
chat_german("Was ist Diabetes?")

In [ ]:
chat_german("Wie wird Diabetes behandelt?")

In [ ]:
test_german_system()


In [ ]:
chat_english("What is hypertension?")

#ECG

In [ ]:
ECG_TRUSTED_WEBSITES = {
    "Basic ECG Knowledge": [
        "https://www.mayoclinic.org/tests-procedures/ekg/about/pac-20384983",
        "https://www.webmd.com/heart-disease/electrocardiogram-ekc",
        "https://www.healthline.com/health/electrocardiogram",
        "https://www.medicalnewstoday.com/articles/153847",
        "https://medlineplus.gov/lab-tests/electrocardiogram-ekg-or-ecg/"
    ],

    "ECG Interpretation": [
        "https://www.mayoclinic.org/tests-procedures/ekg/about/pac-20384983",
        "https://www.webmd.com/heart-disease/reading-an-ekg-electrocardiogram",
        "https://www.healthline.com/health/how-to-read-an-ekg",
        "https://www.medicalnewstoday.com/articles/how-to-read-an-ecg"
    ],

    "ECG Abnormalities": [
        "https://www.mayoclinic.org/diseases-conditions/heart-arrhythmia/symptoms-causes/syc-20350668",
        "https://www.webmd.com/heart-disease/atrial-fibrillation/what-is-atrial-fibrillation",
        "https://www.healthline.com/health/atrial-fibrillation",
        "https://www.medicalnewstoday.com/articles/154617",  # AFib
        "https://www.healthline.com/health/heart-attack/ecg-changes"
    ],

    "Clinical ECG Applications": [
        "https://www.mayoclinic.org/diseases-conditions/heart-attack/diagnosis-treatment/drc-20373106",
        "https://www.webmd.com/heart-disease/guide/heart-disease-heart-attacks",
        "https://www.healthline.com/health/heart-disease/cardiac-monitoring",
        "https://www.medicalnewstoday.com/articles/151444"  # Heart attack
    ],

    "ECG Technology and Equipment": [
        "https://www.healthline.com/health/holter-monitor-24h",
        "https://www.mayoclinic.org/tests-procedures/holter-monitor/about/pac-20385039",
        "https://www.webmd.com/heart-disease/holter-monitor",
        "https://medlineplus.gov/lab-tests/holter-monitor/"
    ]
}

In [ ]:
def add_comprehensive_ecg_knowledge():
    """Add comprehensive ECG knowledge from all trusted sources"""
    print("🫀 BUILDING COMPREHENSIVE ECG KNOWLEDGE BASE")
    print("="*60)

    all_ecg_urls = []
    total_categories = len(ECG_TRUSTED_WEBSITES)

    # Collect all URLs
    for category, urls in ECG_TRUSTED_WEBSITES.items():
        print(f"📚 Category: {category} ({len(urls)} sources)")
        all_ecg_urls.extend(urls)

    print(f"\n🌐 Total ECG websites to process: {len(all_ecg_urls)}")
    print(f"📊 Categories covered: {total_categories}")

    # Remove duplicates while preserving order
    unique_urls = list(dict.fromkeys(all_ecg_urls))
    print(f"🔄 Unique URLs after deduplication: {len(unique_urls)}")

    print("\n🚀 Starting comprehensive ECG data scraping...")
    success = add_websites_to_knowledge_base(unique_urls)

    if success:
        print("\n✅ ECG KNOWLEDGE BASE COMPLETE!")
        print("🧠 Your chatbot now knows about:")
        print("   • ECG basics and procedures")
        print("   • ECG interpretation and reading")
        print("   • Common ECG abnormalities")
        print("   • Clinical applications")
        print("   • ECG equipment and technology")
        return True
    else:
        print("❌ ECG knowledge base building failed")
        return False

def add_ecg_basics():
    """Add basic ECG knowledge (faster option)"""
    basic_urls = ECG_TRUSTED_WEBSITES["Basic ECG Knowledge"]
    print(f"🫀 Adding Basic ECG Knowledge ({len(basic_urls)} sources)...")
    return add_websites_to_knowledge_base(basic_urls)

def add_ecg_interpretation():
    """Add ECG interpretation knowledge"""
    interpretation_urls = ECG_TRUSTED_WEBSITES["ECG Interpretation"]
    print(f"📊 Adding ECG Interpretation Knowledge ({len(interpretation_urls)} sources)...")
    return add_websites_to_knowledge_base(interpretation_urls)

def add_ecg_abnormalities():
    """Add ECG abnormalities knowledge"""
    abnormality_urls = ECG_TRUSTED_WEBSITES["ECG Abnormalities"]
    print(f"⚠️ Adding ECG Abnormalities Knowledge ({len(abnormality_urls)} sources)...")
    return add_websites_to_knowledge_base(abnormality_urls)

In [ ]:
def chat_ecg_german(question):
    """Specialized German ECG chat function"""
    print(f"🫀🇩🇪 EKG-Frage: {question}")
    print("="*60)

    try:
        # Translate German to English
        english_question, detected_lang = rag_system.translate_text(question, 'en')
        print(f"📝 Übersetzt zu Englisch: {english_question}")

        # Search for ECG-specific content with higher result count
        relevant_docs = rag_system.search_documents(english_question, k=7)

        if not relevant_docs:
            german_response = """Ich habe keine spezifischen EKG-Informationen zu dieser Frage in meiner Wissensbasis.

🫀 EKG-Empfehlungen:
• Konsultieren Sie einen Kardiologen
• Besuchen Sie vertrauenswürdige kardiologische Websites
• Lassen Sie sich von einem EKG-Spezialisten beraten

⚠️ EKG-Interpretation sollte nur von qualifizierten medizinischen Fachkräften durchgeführt werden."""

            print(f"🤖 Deutsche EKG-Antwort: {german_response}")
            return german_response

        # Analyze sources for ECG content
        ecg_sources = []
        general_sources = []
        context = ""

        for doc in relevant_docs:
            context += f"{doc['text'][:350]}...\n\n"

            # Check if source is ECG-related
            source_lower = doc['source'].lower()
            if any(term in source_lower for term in ['ekg', 'ecg', 'electrocardiogram', 'heart', 'cardiac', 'arrhythmia']):
                if doc['source'].endswith('.pdf') and '_' in doc['source']:
                    # Website source
                    parts = doc['source'].replace('.pdf', '').split('_')
                    domain = parts[-1]
                    ecg_sources.append(f"EKG-Website: {domain}")
                else:
                    # PDF source
                    ecg_sources.append(f"EKG-PDF: {doc['source']}")
            else:
                general_sources.append(f"Medizin: {doc['source']}")

        # Build English response with ECG focus
        english_response = f"""Based on ECG/EKG medical information about '{question}':

{context[:900]}

This ECG information is for educational purposes only. ECG interpretation should only be performed by qualified medical professionals. Always consult a cardiologist for ECG-related concerns."""

        # Translate to German
        print(f"📝 Übersetze EKG-Antwort ins Deutsche...")
        german_response, _ = rag_system.translate_text(english_response, 'de')

        # Add German sources
        all_sources = ecg_sources + general_sources
        if all_sources:
            sources_text = "\n\n🫀 EKG-Quellen:\n"
            for source in set(all_sources):
                sources_text += f"• {source}\n"
            german_response += sources_text

        # Add ECG-specific disclaimer
        german_response += "\n\n⚠️ WICHTIG: EKG-Interpretation erfordert medizinische Fachkenntnisse. Diese Informationen ersetzen keine professionelle kardiologische Beratung."

        print(f"🤖 Deutsche EKG-Antwort: {german_response}")
        return german_response

    except Exception as e:
        error_msg = "Entschuldigung, es gab einen Fehler bei der EKG-Analyse. Bitte konsultieren Sie einen Kardiologen direkt."
        print(f"🤖 EKG-Fehler-Antwort: {error_msg}")
        return error_msg

def chat_ecg_english(question):
    """Specialized English ECG chat function - FIXED VERSION"""
    print(f"🫀🇺🇸 ECG Question: {question}")
    print("="*60)

    # Search for ECG-specific content
    relevant_docs = rag_system.search_documents(question, k=7)

    if not relevant_docs:
        response = """I don't have specific ECG information about this question in my knowledge base.

🫀 ECG Recommendations:
• Consult a cardiologist
• Visit trusted cardiology websites
• Get professional ECG interpretation

⚠️ ECG interpretation should only be performed by qualified medical professionals."""

        print(f"🤖 ECG Response: {response}")
        return response

    # Analyze ECG relevance
    print("📊 ECG Source Analysis:")
    ecg_specific_count = 0
    general_medical_count = 0

    context = ""
    sources = []

    for doc in relevant_docs:
        context += f"{doc['text'][:350]}...\n\n"
        sources.append(doc['source'])

        # Check ECG relevance
        if any(term in doc['text'].lower() for term in ['ecg', 'ekg', 'electrocardiogram', 'heart rhythm', 'cardiac', 'arrhythmia']):
            ecg_specific_count += 1
        else:
            general_medical_count += 1

    print(f"   🫀 ECG-specific sources: {ecg_specific_count}")
    print(f"   🏥 General medical sources: {general_medical_count}")

    # FIXED: Extract relevance scores separately to avoid nested f-string issue
    relevance_scores = [round(doc['score'], 3) for doc in relevant_docs[:3]]
    print(f"   📈 Relevance scores: {relevance_scores}")

    # Build response
    response = f"""Based on ECG/EKG medical information about '{question}':

{context[:900]}

⚠️ This ECG information is for educational purposes only. ECG interpretation should only be performed by qualified medical professionals. Always consult a cardiologist for ECG-related concerns."""

    # Add sources
    if sources:
        response += f"\n\n🫀 ECG Sources: {', '.join(set(sources))}"

    print(f"🤖 ECG Response: {response}")
    return response


In [ ]:
def test_ecg_knowledge():
    """Test ECG knowledge with comprehensive questions"""
    print("🧪 TESTING ECG KNOWLEDGE BASE")
    print("="*50)

    if len(rag_system.documents) == 0:
        print("⚠️ No documents loaded. Please add ECG knowledge first!")
        print("Run: add_comprehensive_ecg_knowledge()")
        return

    # German ECG test questions
    german_ecg_questions = [
        "Was ist ein EKG?",
        "Wie liest man ein EKG?",
        "Was zeigt ein abnormales EKG?",
        "Was ist Vorhofflimmern im EKG?",
        "Wie erkennt man einen Herzinfarkt im EKG?",
        "Was sind die normalen EKG-Werte?",
        "Welche EKG-Ableitungen gibt es?",
        "Was bedeuten ST-Streckenhebungen?",
        "Wie funktioniert ein Holter-Monitor?",
        "Was ist eine Tachykardie im EKG?"
    ]

    # English ECG test questions
    english_ecg_questions = [
        "What is an ECG?",
        "How do you read an ECG?",
        "What does an abnormal ECG show?",
        "What is atrial fibrillation on ECG?",
        "How to identify heart attack on ECG?",
        "What are normal ECG values?",
        "What are ECG leads?",
        "What do ST elevations mean?",
        "How does a Holter monitor work?",
        "What is tachycardia on ECG?"
    ]

    print(f"🇩🇪 Testing {len(german_ecg_questions)} German ECG questions...")
    print(f"🇺🇸 Testing {len(english_ecg_questions)} English ECG questions...")

    # Test German questions
    for i, question in enumerate(german_ecg_questions[:3], 1):  # Test first 3
        print(f"\n{'='*20} German Test {i} {'='*20}")
        chat_ecg_german(question)

    # Test English questions
    for i, question in enumerate(english_ecg_questions[:3], 1):  # Test first 3
        print(f"\n{'='*20} English Test {i} {'='*20}")
        chat_ecg_english(question)

def ecg_demo_workflow():
    """Complete ECG demo workflow"""
    print("🫀 ECG CHATBOT DEMO WORKFLOW")
    print("="*50)

    print("Step 1: Add ECG knowledge base...")
    success = add_ecg_basics()

    if success:
        print("\nStep 2: Test ECG chatbot...")
        print("\n🇩🇪 German ECG Test:")
        chat_ecg_german("Was ist ein EKG und wofür wird es verwendet?")

        print("\n🇺🇸 English ECG Test:")
        chat_ecg_english("How do you interpret ECG rhythms?")

        print("\n✅ ECG Demo Complete!")
    else:
        print("❌ ECG Demo Failed - check internet connection")

def ecg_quick_start():
    """Quick start guide for ECG functionality"""
    print("""
🫀 ECG CHATBOT QUICK START GUIDE
================================================================

🚀 STEP-BY-STEP SETUP:

1. ADD ECG KNOWLEDGE (Choose one):
   add_ecg_basics()                    # Fast: Basic ECG knowledge
   add_ecg_interpretation()            # Medium: ECG reading skills
   add_comprehensive_ecg_knowledge()   # Complete: All ECG topics

2. TEST ECG CHATBOT:
   test_ecg_knowledge()               # Run comprehensive tests

3. ASK ECG QUESTIONS:
   🇩🇪 German: chat_ecg_german("Was ist ein EKG?")
   🇺🇸 English: chat_ecg_english("What is an ECG?")

================================================================
💡 EXAMPLE ECG QUESTIONS TO TRY:

🇩🇪 GERMAN:
• "Was ist ein EKG?"
• "Wie erkennt man Vorhofflimmern?"
• "Was bedeuten ST-Streckenhebungen?"
• "Wie funktioniert ein Holter-Monitor?"
• "Was sind normale EKG-Werte?"

🇺🇸 ENGLISH:
• "What is an ECG?"
• "How to identify atrial fibrillation?"
• "What do ST elevations mean?"
• "How does a Holter monitor work?"
• "What are normal ECG values?"

================================================================
🔧 ECG KNOWLEDGE CATEGORIES AVAILABLE:
✅ ECG Basics and Procedures
✅ ECG Interpretation and Reading
✅ Common ECG Abnormalities
✅ Clinical ECG Applications
✅ ECG Equipment and Technology

================================================================
⚠️ IMPORTANT ECG DISCLAIMERS:
🏥 ECG interpretation requires medical training
🩺 Always consult qualified cardiologists
📊 This is educational information only
🫀 Never use for actual medical diagnosis
================================================================
""")


In [ ]:
def chat_smart_ecg(question):
    """Smart ECG chat that integrates with the main smart chatbot"""
    print(f"🫀🤖 SMART ECG CHATBOT")
    print(f"Question: {question}")
    print("="*50)

    # Check if question is ECG-related
    ecg_terms = ['ecg', 'ekg', 'electrocardiogram', 'heart rhythm', 'arrhythmia',
                 'atrial fibrillation', 'st elevation', 'holter', 'cardiac monitor',
                 'herzrhythmus', 'vorhofflimmern', 'herz', 'ekg']

    is_ecg_question = any(term in question.lower() for term in ecg_terms)

    if is_ecg_question:
        print("🫀 ECG-related question detected!")
        if bot_config.config["web_search_enabled"]:
            print("🌐 Web search is enabled for enhanced ECG information")
        else:
            print("💡 Tip: Enable ECG web search with enable_ecg_web_search()")

    # Use the smart chatbot with ECG awareness
    try:
        response = chat_smart(question)
        return response
    except NameError:
        # Fallback if smart chatbot not available
        print("⚠️ Smart chatbot not available, using ECG-specific functions")
        # Detect language and use appropriate ECG function
        if any(word in question.lower() for word in ['was', 'wie', 'der', 'die', 'das']):
            return chat_ecg_german(question)
        else:
            return chat_ecg_english(question)

def comprehensive_ecg_demo():
    """Comprehensive demo combining ECG and smart chatbot features"""
    print("🫀🤖 COMPREHENSIVE ECG + SMART CHATBOT DEMO")
    print("="*60)

    # Show current config
    print("1. Current Configuration:")
    try:
        show_chatbot_config()
    except NameError:
        print("   Smart chatbot configuration not available")

    # Test ECG questions with different modes
    ecg_questions = [
        ("Was ist ein EKG?", "German"),
        ("What is atrial fibrillation?", "English"),
        ("Wie erkennt man Herzrhythmusstörungen?", "German"),
        ("How to read ECG waves?", "English")
    ]

    print(f"\n2. Testing {len(ecg_questions)} ECG questions:")
    for question, lang in ecg_questions:
        print(f"\n🫀 {lang} ECG Question: {question}")
        print("-" * 40)
        try:
            response = chat_smart_ecg(question)
            print(f"Preview: {response[:150]}...")
        except Exception as e:
            print(f"Error: {e}")

    print(f"\n✅ Comprehensive ECG demo complete!")

In [ ]:
def test_fixed_ecg():
    """Test the fixed ECG functionality"""
    print("🧪 TESTING FIXED ECG FUNCTIONALITY")
    print("="*50)

    if len(rag_system.documents) == 0:
        print("⚠️ No documents loaded yet.")
        print("📄 Testing with available documents...")

    # Test ECG questions
    print("\n🫀🇺🇸 Testing English ECG Function:")
    chat_ecg_english("What is an ECG?")

    print("\n🫀🇩🇪 Testing German ECG Function:")
    chat_ecg_german("Was ist ein EKG?")

    print("\n🫀🤖 Testing Smart ECG Function:")
    try:
        chat_smart_ecg("How does ECG work?")
    except Exception as e:
        print(f"Smart ECG not available: {e}")

    print("\n✅ ECG functionality tests complete!")

# ================================================================
# ECG SYSTEM INITIALIZATION
# ================================================================

print("""
🫀 ECG-SPECIFIC EXTENSION LOADED! (CORRECTED VERSION)
================================================================

🆕 NEW ECG FEATURES:
✅ Comprehensive ECG website scraping
✅ Specialized ECG chat functions (German/English)
✅ ECG-focused source analysis
✅ ECG abnormality detection knowledge
✅ Clinical ECG application data
✅ ECG equipment and technology info
✅ FIXED: Syntax error in relevance scores
✅ Smart chatbot integration

================================================================
🚀 GET STARTED WITH ECG:
1. ecg_quick_start()                    # Show complete guide
2. add_ecg_basics()                     # Add basic ECG knowledge
3. chat_ecg_german("Was ist ein EKG?")  # Test in German
4. chat_ecg_english("What is an ECG?")  # Test in English
5. chat_smart_ecg("ECG question")       # Smart ECG with auto-detection

================================================================
🧪 TESTING FUNCTIONS:
• test_fixed_ecg()                      # Test all ECG functions
• comprehensive_ecg_demo()              # Complete demo
• ecg_demo_workflow()                   # Step-by-step workflow

================================================================
🎯 RECOMMENDED WORKFLOW:
1. add_comprehensive_ecg_knowledge()    # Build complete ECG database
2. test_fixed_ecg()                     # Test all functions
3. Start asking your ECG questions!

================================================================
⚠️ ECG SAFETY REMINDER:
This chatbot provides ECG education only. ECG interpretation
requires qualified medical professionals. Always consult a
cardiologist for real ECG analysis and diagnosis.
================================================================
""")

# Show the quick start guide
ecg_quick_start()

# Auto-test the fixed functionality
print("\n🧪 Auto-testing fixed ECG functionality...")
test_fixed_ecg()

In [ ]:
chat_ecg_english("How do you interpret ECG rhythms?")

In [ ]:
chat_ecg_german("Was ist ein EKG?")

#OCR

## Auto-Rotation

In [ ]:
# Install paddlepaddle first
!pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple

# Install paddleocr with its dependencies
!pip install paddleocr -i https://pypi.tuna.tsinghua.edu.cn/simple

# Now, import the necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR # This import should now work
import math
from scipy.ndimage import rotate
import os

In [ ]:
# Auto-Rotation and Deskewing for Documents
# Simple, focused solution for straightening scanned documents

import cv2
import numpy as np
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR
import math
from scipy.ndimage import rotate
import os

class DocumentRotationCorrector:
    def __init__(self):
        self.ocr = PaddleOCR(lang='en')

    def detect_text_angle_hough(self, image):
        """Detect text rotation angle using Hough line detection"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)

        # Edge detection
        edges = cv2.Canny(blurred, 50, 150, apertureSize=3)

        # Detect lines using Hough transform
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=80, minLineLength=50, maxLineGap=10)

        if lines is not None and len(lines) > 0:
            angles = []

            for line in lines:
                x1, y1, x2, y2 = line[0]

                # Calculate angle of the line
                angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

                # Normalize angle to -45 to 45 degrees
                if angle > 45:
                    angle = angle - 90
                elif angle < -45:
                    angle = angle + 90

                # Only consider reasonable text angles
                if abs(angle) < 45:
                    angles.append(angle)

            if angles:
                # Use median for robustness against outliers
                detected_angle = np.median(angles)
                return detected_angle

        return 0.0

    def detect_text_angle_projection(self, image):
        """Detect text angle using horizontal projection method"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply threshold to get binary image
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        angles_to_test = np.arange(-15, 16, 0.5)  # Test angles from -15 to 15 degrees
        max_projection = 0
        best_angle = 0

        h, w = binary.shape

        for angle in angles_to_test:
            # Rotate the image slightly
            rotated = rotate(binary, angle, reshape=False, cval=0)

            # Calculate horizontal projection (sum of white pixels in each row)
            projection = np.sum(rotated, axis=1)

            # Calculate variance of projection (higher variance = better alignment)
            variance = np.var(projection)

            if variance > max_projection:
                max_projection = variance
                best_angle = angle

        return best_angle

    def rotate_image(self, image, angle):
        """Rotate image by given angle"""
        if abs(angle) < 0.1:  # Skip rotation if angle is negligible
            return image

        # Get image dimensions
        h, w = image.shape[:2]

        # Calculate rotation center
        center = (w // 2, h // 2)

        # Get rotation matrix
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

        # Calculate new bounding dimensions
        cos_angle = abs(rotation_matrix[0, 0])
        sin_angle = abs(rotation_matrix[0, 1])

        new_w = int((h * sin_angle) + (w * cos_angle))
        new_h = int((h * cos_angle) + (w * sin_angle))

        # Adjust rotation matrix to prevent cropping
        rotation_matrix[0, 2] += (new_w / 2) - center[0]
        rotation_matrix[1, 2] += (new_h / 2) - center[1]

        # Apply rotation
        rotated = cv2.warpAffine(image, rotation_matrix, (new_w, new_h),
                               flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

        return rotated

    def auto_rotate_document(self, image_path, method='combined', show_steps=True):
        """
        Automatically detect and correct document rotation

        Args:
            image_path: Path to the input image
            method: 'hough', 'projection', or 'combined'
            show_steps: Whether to display processing steps
        """
        # Load image
        if isinstance(image_path, str):
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Could not load image: {image_path}")
        else:
            image = image_path  # Already loaded image

        original = image.copy()
        h, w = image.shape[:2]

        print(f"🔄 Auto-rotating document...")
        print(f"📏 Original size: {w}x{h}")

        # Detect rotation angle using different methods
        if method == 'hough':
            angle = self.detect_text_angle_hough(image)
            print(f"📐 Hough method detected angle: {angle:.2f}°")
        elif method == 'projection':
            angle = self.detect_text_angle_projection(image)
            print(f"📐 Projection method detected angle: {angle:.2f}°")
        else:  # combined
            angle1 = self.detect_text_angle_hough(image)
            angle2 = self.detect_text_angle_projection(image)

            # Use the angle with smaller absolute value (more conservative)
            if abs(angle1) <= abs(angle2):
                angle = angle1
                print(f"📐 Combined method: Hough={angle1:.2f}°, Projection={angle2:.2f}°")
                print(f"📐 Selected angle: {angle:.2f}° (Hough)")
            else:
                angle = angle2
                print(f"📐 Combined method: Hough={angle1:.2f}°, Projection={angle2:.2f}°")
                print(f"📐 Selected angle: {angle:.2f}° (Projection)")

        # Apply rotation
        if abs(angle) > 0.1:
            rotated = self.rotate_image(image, angle)
            print(f"✅ Document rotated by {angle:.2f}°")
        else:
            rotated = image
            print(f"✅ No rotation needed (angle: {angle:.2f}°)")

        # Show results
        if show_steps:
            self.show_rotation_results(original, rotated, angle)

        return {
            'original': original,
            'rotated': rotated,
            'angle': angle
        }

    def show_rotation_results(self, original, rotated, angle):
        """Display before/after rotation results"""
        plt.figure(figsize=(15, 6))

        # Original image
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title('Original Document')
        plt.axis('off')

        # Rotated image
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
        plt.title(f'Auto-Rotated (by {angle:.2f}°)')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    def save_rotated_document(self, result, output_path='rotated_document.jpg'):
        """Save the rotated document"""
        cv2.imwrite(output_path, result['rotated'])
        print(f"💾 Rotated document saved as: {output_path}")
        return output_path

    def test_ocr_improvement(self, image_path):
        """Test OCR on original vs rotated image to show improvement"""
        # Auto-rotate the document
        result = self.auto_rotate_document(image_path, method='combined', show_steps=True)

        # Save rotated image
        rotated_path = self.save_rotated_document(result)

        print("\n" + "="*60)
        print("🔍 TESTING OCR IMPROVEMENT")
        print("="*60)

        # Test OCR on original image
        print("\n📄 OCR on ORIGINAL image:")
        print("-" * 40)
        original_texts = self.extract_text(image_path)

        # Test OCR on rotated image
        print(f"\n📄 OCR on ROTATED image:")
        print("-" * 40)
        rotated_texts = self.extract_text(rotated_path)

        # Compare results
        print(f"\n📊 COMPARISON:")
        print(f"   Original: {len(original_texts)} text elements detected")
        print(f"   Rotated:  {len(rotated_texts)} text elements detected")

        if len(rotated_texts) > len(original_texts):
            print(f"   ✅ Improvement: +{len(rotated_texts) - len(original_texts)} more text elements detected!")
        elif len(rotated_texts) == len(original_texts):
            print(f"   ➡️  Same number of elements detected")
        else:
            print(f"   ⚠️  Detected fewer elements in rotated image")

        return result, original_texts, rotated_texts

    def extract_text(self, image_path):
        """Extract text using OCR"""
        try:
            result = self.ocr.predict(image_path)

            if result and len(result) > 0:
                ocr_result = result[0]
                if hasattr(ocr_result, 'get') and 'rec_texts' in ocr_result:
                    texts = ocr_result.get('rec_texts', [])
                    scores = ocr_result.get('rec_scores', [])

                    # Filter high confidence text
                    good_texts = []
                    for i, text in enumerate(texts):
                        score = scores[i] if i < len(scores) else 1.0
                        if score > 0.6:  # Good confidence threshold
                            good_texts.append(text.strip())
                            print(f"   {text.strip()}")

                    return good_texts
        except Exception as e:
            print(f"   ❌ OCR Error: {e}")

        return []

# Convenience functions for easy use
def auto_rotate_and_ocr(image_path):
    """
    Complete pipeline: auto-rotate document and extract text

    Args:
        image_path: Path to the document image

    Returns:
        dict: Contains rotation results and extracted text
    """
    corrector = DocumentRotationCorrector()
    result, original_texts, rotated_texts = corrector.test_ocr_improvement(image_path)

    return {
        'rotation_result': result,
        'original_texts': original_texts,
        'rotated_texts': rotated_texts,
        'improvement': len(rotated_texts) - len(original_texts)
    }

def quick_rotate(image_path, method='combined', output_path='rotated_document.jpg'):
    """
    Quick rotation without OCR testing

    Args:
        image_path: Path to input image
        method: 'hough', 'projection', or 'combined'
        output_path: Path to save rotated image

    Returns:
        dict: Rotation results
    """
    corrector = DocumentRotationCorrector()
    result = corrector.auto_rotate_document(image_path, method=method, show_steps=True)
    corrector.save_rotated_document(result, output_path)
    return result

def batch_rotate(image_paths):
    """
    Rotate multiple images

    Args:
        image_paths: List of image paths
    """
    corrector = DocumentRotationCorrector()
    results = []

    for i, path in enumerate(image_paths):
        print(f"\n🔄 Processing image {i+1}/{len(image_paths)}: {path}")
        try:
            result = corrector.auto_rotate_document(path, method='combined', show_steps=False)
            output_path = f'rotated_document_{i+1}.jpg'
            corrector.save_rotated_document(result, output_path)
            results.append({'input': path, 'output': output_path, 'angle': result['angle']})
        except Exception as e:
            print(f"❌ Error processing {path}: {e}")
            results.append({'input': path, 'output': None, 'error': str(e)})

    return results

# For Google Colab
def colab_auto_rotate():
    """Upload and auto-rotate document in Google Colab"""
    try:
        from google.colab import files

        print("📁 Upload your document image:")
        uploaded = files.upload()

        for filename in uploaded.keys():
            print(f"\n🔄 Processing {filename}...")

            # Auto-rotate and test OCR
            result = auto_rotate_and_ocr(filename)

            print(f"\n📈 FINAL RESULTS for {filename}:")
            print(f"   Rotation angle: {result['rotation_result']['angle']:.2f}°")
            print(f"   OCR improvement: {result['improvement']} more text elements detected")

            # Download the rotated image
            if os.path.exists('rotated_document.jpg'):
                files.download('rotated_document.jpg')
                print("📥 Rotated document downloaded!")

    except ImportError:
        print("❌ This function is for Google Colab only")


result = quick_rotate('/content/1_rvqgsUk19Wi50QsIyvt94Q.webp')
